***GENERATED CODE FOR crimecomparisonregression PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Case Number', 'transformation_label': 'String Indexer'}], 'feature': 'Case Number', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'JD413738', 'max': 'JE295084', 'missing': '0', 'distinct': '500'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Case Number'}, {'feature_label': 'Case Number', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Case Number')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Date', 'transformation_label': 'String Indexer'}], 'feature': 'Date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1/1/2008 2:00', 'max': '9/15/2020 12:00', 'missing': '0', 'distinct': '498'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Date'}, {'feature_label': 'Date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Block', 'transformation_label': 'String Indexer'}], 'feature': 'Block', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '0000X E 102ND ST', 'max': '130XX S DREXEL AVE', 'missing': '0', 'distinct': '483'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Block'}, {'feature_label': 'Block', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Block')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'IUCR', 'transformation_label': 'String Indexer'}], 'feature': 'IUCR', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '1108.25', 'stddev': '834.87', 'min': '031A', 'max': '930', 'missing': '0', 'distinct': '81'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'IUCR'}, {'feature_label': 'IUCR', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('IUCR')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Primary Type', 'transformation_label': 'String Indexer'}], 'feature': 'Primary Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'ASSAULT', 'max': 'WEAPONS VIOLATION', 'missing': '0', 'distinct': '19'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Primary Type'}, {'feature_label': 'Primary Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Primary Type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Description', 'transformation_label': 'String Indexer'}], 'feature': 'Description', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '$500 AND UNDER', 'max': 'VIOLATION OF CIVIL NO CONTACT ORDER', 'missing': '0', 'distinct': '84'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Description'}, {'feature_label': 'Description', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Description')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Location Description', 'transformation_label': 'String Indexer'}], 'feature': 'Location Description', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '495', 'mean': '', 'stddev': '', 'min': 'AIRPORT BUILDING NON-TERMINAL - NON-SECURE AREA', 'max': 'VEHICLE NON-COMMERCIAL', 'missing': '31', 'distinct': '55'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Location Description'}, {'feature_label': 'Location Description', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Location Description')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FBI Code', 'transformation_label': 'String Indexer'}], 'feature': 'FBI Code', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '11.77', 'stddev': '6.42', 'min': '04A', 'max': '7', 'missing': '0', 'distinct': '17'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FBI Code'}, {'feature_label': 'FBI Code', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FBI Code')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Updated On', 'transformation_label': 'String Indexer'}], 'feature': 'Updated On', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1/13/2021 15:39', 'max': '9/9/2020 15:41', 'missing': '0', 'distinct': '221'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Updated On'}, {'feature_label': 'Updated On', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Updated On')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Location', 'transformation_label': 'String Indexer'}], 'feature': 'Location', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '486', 'mean': '', 'stddev': '', 'min': '(41.656458782, -87.537470986)', 'max': '(42.02117847, -87.672359309)', 'missing': '177', 'distinct': '483'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Location'}, {'feature_label': 'Location', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Location')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run crimecomparisonregressionHooks.ipynb
try:
	#sourcePreExecutionHook()

	chicagopanddcrimeratecomparison = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Chicago_P_and_D_Crime_Rate_Comparison.csv', 'filename': 'Chicago_P_and_D_Crime_Rate_Comparison.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(chicagopanddcrimeratecomparison)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run crimecomparisonregressionHooks.ipynb
try:
	#transformationPreExecutionHook()

	crimecomparisonregressionautofe = TransformationMain.run(chicagopanddcrimeratecomparison,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "ID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "12314882.29", "stddev": "59404.08", "min": "12206626", "max": "12418133", "missing": "0"}, "updatedLabel": "ID"}, {"transformationsData": [{"feature_label": "Case Number", "transformation_label": "String Indexer"}], "feature": "Case Number", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "JD413738", "max": "JE295084", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Case Number"}, {"transformationsData": [{"feature_label": "Date", "transformation_label": "String Indexer"}], "feature": "Date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1/1/2008 2:00", "max": "9/15/2020 12:00", "missing": "0", "distinct": "498"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Date"}, {"transformationsData": [{"feature_label": "Block", "transformation_label": "String Indexer"}], "feature": "Block", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "0000X E 102ND ST", "max": "130XX S DREXEL AVE", "missing": "0", "distinct": "483"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Block"}, {"transformationsData": [{"feature_label": "IUCR", "transformation_label": "String Indexer"}], "feature": "IUCR", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "1108.25", "stddev": "834.87", "min": "031A", "max": "930", "missing": "0", "distinct": "81"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "IUCR"}, {"transformationsData": [{"feature_label": "Primary Type", "transformation_label": "String Indexer"}], "feature": "Primary Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "ASSAULT", "max": "WEAPONS VIOLATION", "missing": "0", "distinct": "19"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Primary Type"}, {"transformationsData": [{"feature_label": "Description", "transformation_label": "String Indexer"}], "feature": "Description", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "$500 AND UNDER", "max": "VIOLATION OF CIVIL NO CONTACT ORDER", "missing": "0", "distinct": "84"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Description"}, {"transformationsData": [{"feature_label": "Location Description", "transformation_label": "String Indexer"}], "feature": "Location Description", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "495", "mean": "", "stddev": "", "min": "AIRPORT BUILDING NON-TERMINAL - NON-SECURE AREA", "max": "VEHICLE NON-COMMERCIAL", "missing": "31", "distinct": "55"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Location Description"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Arrest", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.12", "stddev": "0.33", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Arrest"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Domestic", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.27", "stddev": "0.44", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Domestic"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Beat", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1187.82", "stddev": "738.33", "min": "111", "max": "2535", "missing": "0"}, "updatedLabel": "Beat"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "District", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "11.65", "stddev": "7.38", "min": "1", "max": "25", "missing": "0"}, "updatedLabel": "District"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Ward", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "23.57", "stddev": "13.9", "min": "1", "max": "50", "missing": "0"}, "updatedLabel": "Ward"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Community Area", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "37.49", "stddev": "21.93", "min": "1", "max": "77", "missing": "0"}, "updatedLabel": "Community Area"}, {"transformationsData": [{"feature_label": "FBI Code", "transformation_label": "String Indexer"}], "feature": "FBI Code", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "11.77", "stddev": "6.42", "min": "04A", "max": "7", "missing": "0", "distinct": "17"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FBI Code"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "X Coordinate", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "486", "mean": "1165076.47", "stddev": "17659.42", "min": "1100658", "max": "1202552", "missing": "14"}, "updatedLabel": "X Coordinate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Y Coordinate", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "323", "mean": "1885447.09", "stddev": "31324.11", "min": "1818786", "max": "1948756", "missing": "14"}, "updatedLabel": "Y Coordinate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2018.68", "stddev": "2.73", "min": "2001", "max": "2020", "missing": "0"}, "updatedLabel": "Year"}, {"transformationsData": [{"feature_label": "Updated On", "transformation_label": "String Indexer"}], "feature": "Updated On", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1/13/2021 15:39", "max": "9/9/2020 15:41", "missing": "0", "distinct": "221"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Updated On"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Latitude", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "486", "mean": "41.84", "stddev": "0.09", "min": "41.65645878", "max": "42.02117847", "missing": "177"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Latitude"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Longitude", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "323", "mean": "-87.67", "stddev": "0.06", "min": "-87.84004995", "max": "-87.53528009", "missing": "14"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Longitude"}, {"transformationsData": [{"feature_label": "Location", "transformation_label": "String Indexer"}], "feature": "Location", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "486", "mean": "", "stddev": "", "min": "(41.656458782, -87.537470986)", "max": "(42.02117847, -87.672359309)", "missing": "177", "distinct": "483"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Location"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PandemicPeriod", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.99", "stddev": "0.11", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "PandemicPeriod"}]}))

	#transformationPostExecutionHook(crimecomparisonregressionautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run crimecomparisonregressionHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(crimecomparisonregressionautofe, ["ID", "Arrest", "Domestic", "Beat", "District", "Ward", "Community Area", "X Coordinate", "Y Coordinate", "Year", "Latitude", "Longitude", "Case Number_stringindexer", "Date_stringindexer", "Block_stringindexer", "IUCR_stringindexer", "Primary Type_stringindexer", "Description_stringindexer", "Location Description_stringindexer", "FBI Code_stringindexer", "Updated On_stringindexer", "Location_stringindexer"], "PandemicPeriod")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

